# First Part

In [340]:
import csv , os, glob , random, time , codecs
import pandas as pd
import requests, urllib
from bs4 import BeautifulSoup
import requests as rq
import time
import random

url_1 = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html'
url_2 = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html'
url_3 = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html'

# collect all of the urls in a list
url_list = []
for url in  [url_1 , url_2 , url_3]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    for link in soup.find_all('a'):
        url_list.append(link.get('href'))

Downloading the raw html files and save them 

In [339]:
for i ,link in enumerate(url_list):
    try:
        response = rq.get(link)
    except rq.exceptions.RequestException as e:
        E.append(e)
        time.sleep(random.choice(range(1,4)))
        response = rq.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    f = open('article_'+str(i)+'.html','w')
    f.write(str(soup.encode('ascii').decode('ascii')))
    f.close()
    time.sleep(random.choice(range(1,4)))

0
1
2
3
4


Function to only collect the plot and intro

In [4]:
def plot_text(soup):
    start = soup.find('h2').find_next('p')
    plot = start.text.strip()
    for elem in start.next_siblings:
        if elem.name != 'p':
            break
        plot += elem.text.strip()
    return plot

def intro_text(soup):
    start = soup.find('p')
    intro = start.text.strip()
    for elem in start.next_siblings:
        if elem.name != 'p':
            break
        intro += elem.text.strip()
    return intro


In [84]:
columns =['Wikipedia Url', 'title', 'intro', 'plot','Directed by','Produced by','Written by','Screenplay by','Story by','Based on','Starring','Narrated by','Music by','Cinematography','Edited by','Production','company','Distributed by','Release date','Running time','Country','Language','Budget','Box','office']
df = pd.DataFrame(columns=columns)

Extract the parts we needed and put the n arrange

In [ ]:
htmlfiles = glob.glob(os.path.join(mydir, '*.html'))

E = []  # if a link can not be opened it would save in this list
j = 0
for link in htmlfiles:
    try:
        html_wiki  = open(link , 'r')
        soup = BeautifulSoup(html_wiki ,'html.parser')
        
        df.loc[j , 'title'] = soup.find('h1').get_text()
        df.loc[j ,'intro'] =  intro_text(soup)
        if df.loc[j ,'intro']=='':
            try:
                df.loc[j ,'intro'] =soup.find('p').find_next_sibling('p').text.strip()
            except:
                pass    
        
        
        for x,y in  zip(soup.find('table', class_="infobox vevent").find_all('th') , soup.find('table', class_="infobox vevent").find_all('td')):
                if x.text.strip() in columns:
                    df.loc[j , x.text.strip()] = y.text.strip()
        try:
            df.loc[j ,'plot'] = plot_text(soup)
            
             
        except:
            pass
        
    except :
         E.append(link)
    j+=1 

write the datfarme into tsv files

In [ ]:
# write the datfarme into tsv files
for k in range(df.shape[0]):
    
    output = os.path.join('wiki_Articles_tvs' , 'article_' + str(k) + '.tsv')
    with open(output, 'wt' , encoding='ascii') as out_file:
        tsv_writer = csv.writer(out_file, delimiter ='\t' )
        tsv_writer.writerow(df.loc[k])

In [201]:
# reading the writen files in tsv
#pd.read_csv('wiki_article.tsv',sep='\t',encoding='utf-8')  index_col='Name'
import codecs    
f = codecs.open('wiki_article.tsv' )
f_text = f.read()
f.close()

# Part 2

In [206]:
import pandas as pd
import numpy as np
from stop_words import get_stop_words
from nltk.tokenize import RegexpTokenizer
import re
import nltk
import nltk.corpus
import gensim
import nltk; nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [207]:
df_h = df
# Cleaing the text
for col in df.columns.tolist()[1:]:
    df[col]= df[col].map(lambda x: x.lower())
    df[col]= df[col].map(lambda x: re.sub('[,\'\"\;\(\)\:\,\“\”\-_\%\.!?\]\]\t]', '', str(x)))
    df[col]= df[col].map(lambda x: re.sub('[^\sA-Za-z0-9]', '', str(x)))
    df[col]= df[col].map(lambda x: re.sub(r"\s\s+", "", x))

In [331]:
df

,Wikipedia Url,title,intro,plot,Directed by,Produced by,Written by,Screenplay by,Story by,Based on,...,Production,company,Distributed by,Release date,Running time,Country,Language,Budget,Box,office
0,https://en.wikipedia.org/wiki/Z.P.G.,zpg,zpg short for zero population growth is a 1972...,set in the future the earth has become severel...,michael campus,frank de felittamax simon ehrlichtom madigan,frank de felittamax simon ehrlich,nan,nan,nan,...,nan,nan,scotiabarber,25 may 1972 19720525,97 minutes,denmarkunited states1,english,nan,nan,nan
1,https://en.wikipedia.org/wiki/Zee_and_Co.,zee and co,zee and co also known as x y and zee and zee a...,zee blakely elizabeth taylor is a loud coarse ...,brian g hutton,elliot kastnerjay kanteralan ladd jr,edna obrien,nan,nan,nan,...,nan,nan,nan,january 21 1972,110 minutes,united kingdom,english,nan,nan,nan
2,https://en.wikipedia.org/wiki/And_Now_the_Scre...,and now the screaming starts,and now the screaming starts is a 1973 british...,in 1795 newlyweds catherine beacham and charle...,roy ward baker,max rosenbergmilton subotskygustave berne,nan,roger marshall,nan,fengriffen novellaby david case,...,nan,nan,foxrank uk cinerama releasing corporation us,april 27 1973,91 minutes,united kingdom,english,5000001,nan,nan
3,https://en.wikipedia.org/wiki/The_Asphyx,the asphyx,the asphyx is a 1972 british horror film1 dire...,in victorian england philanthropic scientist s...,peter newbrook,john brittany,christina beers storylaurence beers storybrian...,nan,nan,nan,...,nan,nan,cinema epochunited entertainment,1972 1972,99 minutes,united kingdom,english,nan,nan,nan
4,https://en.wikipedia.org/wiki/Assassin_(1973_f...,assassin 1973 film,assassin is a 1973 british thriller film direc...,the british government hires an assassin to ki...,peter cranedavid m jackson,michael sloane,ian hendryedward juddfrank windsorray brooks,nan,nan,nan,...,nan,nan,nan,83 minutes,united kingdom,english,nan,nan,nan,nan
5,https://en.wikipedia.org/wiki/The_Blockhouse,the blockhouse,the blockhouse is a 1973 film based on a 1955 ...,on dday a mixed group of forced labourers held...,clive rees,edgar bronfman jrantony rufus isaacs,nan,john gouldclive rees,nan,novel le blockhaus by jeanpaul clbert,...,nan,nan,metrogoldwynmayer mgmcannon films,1973,93 minutes,united kingdom,english,nan,nan,nan
6,https://en.wikipedia.org/wiki/Carry_On_Girls,carry on girls,carry on girls is the 25th in the series of ca...,the seaside town of fircombe is facing a crisi...,gerald thomas,peter rogers,talbot rothwell,nan,nan,nan,...,nan,nan,rank organisation,9 november 1973,88 mins,united kingdom,english,205962,nan,nan
7,https://en.wikipedia.org/wiki/Charley_One-Eye,charley oneeye,charley oneeye is a 1973 britishamerican film ...,a black union army deserter and his crippled a...,don chaffey1,david frostjames swan,keith leonard,nan,nan,nan,...,nan,nan,paramount pictures,april 18 1973,96 minutes,united statesunited kingdom,english,nan,nan,nan
8,https://en.wikipedia.org/wiki/Commuter_Husbands,commuter husbands,commuter husbands is a 1972 comedy film by not...,the story teller drake enters the penthouse cl...,derek ford,morton lewis,derek ford,nan,nan,nan,...,nan,nan,scotia international,1972 1972,85 minutes,united kingdom,english,nan,nan,nan
9,https://en.wikipedia.org/wiki/The_Creeping_Flesh,the creeping flesh,the creeping flesh is a 1973 british horror fi...,prof emmanuel hildern peter cushing a victoria...,freddie francis,michael p redbourn,peter spenceleyjonathan rumbold,nan,nan,nan,...,nan,nan,tigon british film productions ukcolumbia pict...,1973 united kingdomfebruary 12 1973 united states,94 minutes,united kingdom,english,nan,nan,nan


In [128]:
stop_words = set(nltk.corpus.stopwords.words('english'))

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [323]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

In [330]:
lemmatizer.lemmatize("gsasaso")

'gsasaso'

In [223]:
df2 =df[['intro' , 'plot']]
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stemm(text):
    stm = []
    for word in text:
        stm.append(ps.stem(word))
    return stm    

In [141]:
for col in ['intro' , 'plot']:
    df2[col] =df2[col].map(lambda x:remove_stopwords(x))
    
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [268]:
stop_words = set(nltk.corpus.stopwords.words('english'))
txt = ''
for i in df.index:
    txt += ''.join(df.loc[i , ['intro' , 'plot']])

txxt = [word for word in txt.split(' ') if word not in stop_words]   
txxt = stemm(txxt)
txxt = set(txxt)

In [269]:
new_text = []
for word in txxt:
    if word not in stop_words:
        new_text.append(word)

In [259]:
doc_num = {}
for i in df.index:
    doc_num[i] = df.loc[i, 'intro'].split(' ') + df.loc[i,'plot'].split(' ')
    
#df[['intro' ,'plot']].apply(lambda x,y: x.split(' '), y.split(' ') )

In [270]:
len(new_text)

1837

In [291]:

inverted_index

,0,2894,2,95,2450,1288,6,7,1510,9,...,3612,3587,3599,3600,3621,3622,3625,3626,3628,3629
0,0,0,0,NaN,0,0,0,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
inverted_index[2894].dropna()

0      0
13    13
15    15
Name: 2894, dtype: object

In [290]:
inverted_index=pd.DataFrame( index=[*doc_num], columns=[*vocabulary.values()] )
for i in doc_num.keys():
    for w in vocabulary.keys():
        if w  in doc_num[i]:
             inverted_index.loc[i, vocabulary[w]] = i

In [322]:
inverted_index.apply(lambda x : x.dropna().tolist(), axis=0).to_dict()

{0: [0],
 2894: [0, 13, 15],
 2: [0],
 95: [],
 2450: [0, 12],
 1288: [0, 1, 3, 8],
 6: [0],
 7: [0],
 1510: [],
 9: [0],
 3617: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 3552: [],
 3263: [0, 1, 11, 14, 16],
 13: [],
 3445: [2],
 2126: [],
 140: [0],
 143: [],
 144: [0],
 19: [],
 20: [],
 21: [],
 22: [0],
 2462: [0],
 2125: [0, 11],
 26: [0],
 27: [0],
 3120: [],
 67: [],
 55: [],
 78: [0],
 319: [0, 1],
 3331: [0, 11, 12, 16],
 2748: [],
 560: [],
 37: [],
 38: [0],
 39: [0],
 2516: [0, 10, 13],
 42: [0],
 44: [0],
 1968: [],
 46: [0],
 975: [],
 3273: [0, 16],
 50: [],
 51: [0],
 52: [0],
 53: [],
 54: [],
 3029: [],
 2004: [],
 256: [],
 1487: [],
 3356: [0, 17],
 3045: [0],
 63: [],
 64: [],
 65: [],
 68: [],
 69: [],
 70: [],
 919: [],
 72: [0],
 73: [0],
 74: [],
 75: [0],
 1596: [],
 77: [0],
 80: [],
 81: [0],
 82: [0],
 2588: [0, 13],
 84: [],
 85: [0],
 87: [0],
 88: [0],
 89: [],
 365: [1],
 125: [0],
 2693: [],
 93: [],
 98: [],
 2122: [0, 9

In [299]:
vocabulary

{'zpg': 0,
 'short': 2894,
 'zero': 2,
 'popul': 95,
 'growth': 2450,
 '1972': 1288,
 'danishamerican': 6,
 'dystopian': 7,
 'scienc': 1510,
 'fiction': 9,
 'film': 3617,
 'direct': 3552,
 'michael': 3263,
 'campu': 13,
 'star': 3445,
 'oliv': 2126,
 'reed': 140,
 'geraldin': 143,
 'chaplin': 144,
 'inspir': 19,
 'nonfict': 20,
 'bestsel': 21,
 'book': 22,
 'bomb': 2462,
 'paul': 2125,
 'r': 26,
 'ehrlich': 27,
 'concern': 3120,
 'overpopul': 67,
 'futur': 55,
 'earth': 78,
 'whose': 319,
 'world': 3331,
 'govern': 2748,
 'execut': 560,
 'violat': 37,
 '30year': 38,
 'ban': 39,
 'children': 2516,
 'denmark': 42,
 'almost': 44,
 'entir': 1968,
 'setbound': 46,
 'featur': 975,
 'art': 3273,
 'design': 50,
 'reflect': 51,
 'bleak': 52,
 'oppress': 53,
 'futureset': 54,
 'becom': 3029,
 'sever': 2004,
 'pollut': 256,
 'peopl': 1487,
 'need': 3356,
 'wear': 3045,
 'breath': 63,
 'mask': 64,
 'outsid': 65,
 'affect': 68,
 'avail': 69,
 'resourc': 70,
 'perman': 919,
 'thick': 72,
 'smog': 73

In [271]:
from collections import Counter
Counter(new_text)

Counter({'inherit': 1,
         'wolenski': 1,
         'utterli': 1,
         'bombay': 1,
         'ill': 1,
         'countri': 1,
         'feel': 1,
         'portmanteau': 1,
         'sutherland': 1,
         'solv': 1,
         'drop': 1,
         'centuri': 1,
         'doctor': 1,
         'apolog': 1,
         'hardcore1th': 1,
         'europ': 1,
         'butterworthar': 1,
         'abc': 1,
         'citron': 1,
         'roeg': 1,
         'ree': 1,
         'offic': 1,
         'anoth': 1,
         'lundquist': 1,
         'agre': 1,
         'summer': 1,
         'augusta': 1,
         'bless': 1,
         'game': 1,
         'cush': 1,
         'anddespit': 1,
         'paper': 1,
         'billmainli': 1,
         'coupl': 1,
         'compos': 1,
         'photo': 1,
         'june': 1,
         'obviou': 1,
         'birdi': 1,
         'reed': 1,
         'arriv': 1,
         'expos': 1,
         'previou': 1,
         'need': 1,
         'dubiou': 1,
         '

In [265]:
for i in doc_num.keys():
    if 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [244]:
for x, y in enumerate(new_text):
    for 
    if y in 
    

zpg 0
short 1
zero 2
popul 3
growth 4
1972 5
danishamerican 6
dystopian 7
scienc 8
fiction 9
film 10
direct 11
michael 12
campu 13
star 14
oliv 15
reed 16
geraldin 17
chaplin 18
inspir 19
nonfict 20
bestsel 21
book 22
popul 23
bomb 24
paul 25
r 26
ehrlich 27
film 28
concern 29
overpopul 30
futur 31
earth 32
whose 33
world 34
govern 35
execut 36
violat 37
30year 38
ban 39
children 40
film 41
denmark 42
film 43
almost 44
entir 45
setbound 46
featur 47
art 48
direct 49
design 50
reflect 51
bleak 52
oppress 53
futureset 54
futur 55
earth 56
becom 57
sever 58
pollut 59
peopl 60
need 61
wear 62
breath 63
mask 64
outsid 65
sever 66
overpopul 67
affect 68
avail 69
resourc 70
perman 71
thick 72
smog 73
settl 74
dismal 75
citi 76
cover 77
earth 78
entir 79
surfac 80
animalseven 81
common 82
household 83
petsar 84
extinct 85
peopl 86
eat 87
tasteless 88
brightcolor 89
past 90
plastic 91
contain 92
reduc 93
world 94
popul 95
world 96
govern 97
decre 98
children 99
may 100
conceiv 101
next 102
30 1

alway 1024
rain 1025
noth 1026
tourist 1027
councillor 1028
sidney 1029
fiddler 1030
sid 1031
jame 1032
hit 1033
notion 1034
hold 1035
beauti 1036
contest 1037
mayor 1038
frederick 1039
bumbl 1040
kenneth 1041
connor 1042
taken 1043
idea 1044
feminist 1045
councillor 1046
augusta 1047
prodworthi 1048
june 1049
whitfield 1050
outrag 1051
storm 1052
meet 1053
motion 1054
carri 1055
augusta 1056
absenc 1057
sidney 1058
contact 1059
publicist 1060
peter 1061
potter 1062
bernard 1063
bresslaw 1064
help 1065
organis 1066
sidney 1067
girlfriend 1068
conni 1069
philpott 1070
joan 1071
sim 1072
run 1073
local 1074
hotel 1075
soon 1076
residentsinclud 1077
eccentr 1078
mr 1079
duke 1080
joan 1081
hickson 1082
randi 1083
old 1084
admir 1085
peter 1086
butterworthar 1087
outnumb 1088
put 1089
model 1090
includ 1091
diminut 1092
biker 1093
hope 1094
spring 1095
barbara 1096
windsor 1097
tall 1098
buxom 1099
dawn 1100
brake 1101
margaret 1102
nolan 1103
catfight 1104
orchestr 1105
hope 1106
think 11

seduct 2838
ofmiss 2839
parkyn 2840
wendi 2841
also 2842
appear 2843
surpris 2844
minist 2845
barri 2846
jean 2847
tri 2848
keep 2849
truth 2850
inspector 2851
ruff 2852
search 2853
miss 2854
ovi 2855
wilfr 2856
pott 2857
ancient 2858
antisleaz 2859
mp 2860
stay 2861
temporarili 2862
adjoin 2863
flat 2864
birdi 2865
minist 2866
wife 2867
tri 2868
deal 2869
hippi 2870
utmost 2871
discredit 2872
mainwaringbrown 2873
ovi 2874
natur 2875
caus 2876
end 2877
troubledont 2878
look 2879
italian 2880
venezia 2881
un 2882
dicembr 2883
rosso 2884
shock 2885
1973 2886
independ 2887
film 2888
direct 2889
nicola 2890
roeg 2891
thriller 2892
adapt 2893
short 2894
stori 2895
daphn 2896
du 2897
maurier 2898
juli 2899
christi 2900
donald 2901
sutherland 2902
portray 2903
marri 2904
coupl 2905
travel 2906
venic 2907
follow 2908
recent 2909
accident 2910
death 2911
daughter 2912
husband 2913
accept 2914
commiss 2915
restor 2916
church 2917
encount 2918
two 2919
sister 2920
one 2921
claim 2922
clairvoy 292

In [242]:
# 2.1.1) Create your index!
vocabulary = pd.DataFrame(new_text, columns= ['vocabulary']).reset_index().set_index('vocabulary')['index'].to_dict()

In [256]:
vocabulary['zpg']

0

In [214]:
pd.DataFrame(columns= ['vocabulary'] )
vocabulary = {}
for word in new_text:
    
    vocabulary[word] = 
        

['zpg',
 'short',
 'zero',
 'population',
 'growth',
 '1972',
 'danishamerican',
 'dystopian',
 'science',
 'fiction',
 'film',
 'directed',
 'michael',
 'campus',
 'starring',
 'oliver',
 'reed',
 'geraldine',
 'chaplin',
 'inspired',
 'nonfiction',
 'bestselling',
 'book',
 'population',
 'bomb',
 'paul',
 'r',
 'ehrlich',
 'film',
 'concerns',
 'overpopulated',
 'future',
 'earth',
 'whose',
 'world',
 'government',
 'executes',
 'violate',
 '30year',
 'ban',
 'children',
 'filmed',
 'denmark',
 'film',
 'almost',
 'entirely',
 'setbound',
 'featuring',
 'art',
 'direction',
 'designed',
 'reflect',
 'bleak',
 'oppressive',
 'futureset',
 'future',
 'earth',
 'become',
 'severely',
 'polluted',
 'people',
 'need',
 'wear',
 'breathing',
 'masks',
 'outside',
 'severe',
 'overpopulation',
 'affecting',
 'available',
 'resources',
 'permanent',
 'thick',
 'smog',
 'settled',
 'dismal',
 'cities',
 'cover',
 'earths',
 'entire',
 'surface',
 'animalseven',
 'common',
 'household',
 'pe

In [174]:
unique_words = set(new_text)

In [182]:
'' in txt.split(' ')

True

In [151]:
df.loc[19,'plot']

'in the seventeenth century1 pirate captain ras mohammed peter boyle accompanied by his incompetent irish cook dick scratcher peter sellers and three crewmen buries three chests of treasure on an unnamed island scratcher then kills the captain and three crewmen he returns to the pirate ship and proclaims himself the new captain as only he knows the location of the treasure he soon discovers his predecessors treasure map was drawn in disappearing inkfourteen years later the pirate ship reaches ireland although scratcher believes they are on a greek island the crew behaves gluttonously with the exception of the debonair and cerebral pierre rodriquez anthony franciosathey abduct an irish boy jeremiah richard willis whom scratcher believes can see ghosts and will therefore be able to contact ras mohammeds ghost for directions to the treasure after seeing off the british navy by posing as portuguese fishermen suffering the red plague the pirates sail to algiers where jeremiah is taken priso

In [132]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])


'The Great Train Robbery (1903 film)'

In [310]:
tag_plot = soup.select_one('#Plot').findParent('h2') #.find_next_sibling('p')

In [309]:
 plot_text(tag_plot)

[]

In [315]:
tag_plot.next_sibling.next_sibling

<div class="thumb tleft"><div class="thumbinner" style="width:222px;"><div class="PopUpMediaTransform" id="mwe_player_0" style="width:220px;" videopayload='&lt;div class="mediaContainer" style="width:640px"&gt;&lt;video id="mwe_player_1" poster="//upload.wikimedia.org/wikipedia/commons/thumb/d/d7/The_Great_Train_Robbery_%281903%29.webm/640px--The_Great_Train_Robbery_%281903%29.webm.jpg" controls="" preload="none" autoplay="" style="width:640px;height:480px" class="kskin" data-durationhint="807.708" data-startoffset="0" data-mwtitle="The_Great_Train_Robbery_(1903).webm" data-mwprovider="wikimediacommons"&gt;&lt;source src="//upload.wikimedia.org/wikipedia/commons/transcoded/d/d7/The_Great_Train_Robbery_%281903%29.webm/The_Great_Train_Robbery_%281903%29.webm.480p.vp9.webm" type="video/webm; codecs=&amp;quot;vp9, opus&amp;quot;" data-title="SD VP9 (480P)" data-shorttitle="VP9 480P" data-transcodekey="480p.vp9.webm" data-width="640" data-height="480" data-bandwidth="825672" data-framerate=

In [320]:
def plot_text(soup):
    start = soup.find('h2').find_next('p')
    plot = start.text.strip()
    for elem in start.next_siblings:
        if elem.name != 'p':
            break
        plot += elem.text.strip()
    return plot

In [287]:
a1 = tag_plot.find_next_sibling()
a2 = a1.find_next_sibling()
a3 = a2.find_next_sibling()
a4 = a3.find_next_sibling()
a5 = a4.find_next_sibling()


In [250]:
for s in soup.find('h2').find_next_siblings('p'):
    #if s.name == 'p':
    print(s.text)

In [179]:
plot_text = []
for tag in tag_plot:
    if tag.name == 'p':
        plot_text.append(tag.text[:-2])
    

In [201]:
for tag in tag_plot.find_all('p'):
    print(tag.text)

In [62]:
start = soup.find('p')
intro = start.text.strip()
for elem in start.next_siblings:
    if elem.name != 'p':
        break
    intro += elem.text.strip()
intro

''

In [66]:
start.find_next_sibling('p').text.strip()

'Assassin is a 1973 British thriller film directed by Peter Crane and starring Ian Hendry, Edward Judd and Frank Windsor.[1]'

In [192]:
next1 = tag_plot.find_next_sibling()
print(tag_plot.find_next_sibling())
print(next1.find_next_sibling())
next2 = next1.find_next_sibling()
print(next2.find_next_sibling())
next3 = next2.find_next_sibling()
print(next3.find_next_sibling())

<p>The film opens with two bandits breaking into a railroad telegraph office, where they force the operator at gunpoint to have a train stopped and to transmit orders for the engineer to fill the <a href="/wiki/Locomotive" title="Locomotive">locomotive</a>'s <a href="/wiki/Tender_(rail)" title="Tender (rail)">tender</a> at the station's water tank. They then knock the operator out and tie him up. As the train stops it is boarded by the bandits‍—‌now four. Two bandits enter an <a href="/wiki/Passenger_car_(rail)#Express_car" title="Passenger car (rail)">express car</a>, kill a messenger and open a box of valuables with dynamite; the others kill the <a href="/wiki/Fireman_(steam_engine)" title="Fireman (steam engine)">fireman</a> and force the <a class="mw-redirect" href="/wiki/Engineer_(railroad)" title="Engineer (railroad)">engineer</a> to halt the train and disconnect the locomotive. The bandits then force the passengers off the train and rifle them for their belongings. One passenger

In [97]:
soup.find_all('p')

[<p><i><b>The Great Train Robbery</b></i> is a 1903 American <a href="/wiki/Silent_film" title="Silent film">silent</a> <a href="/wiki/Short_film" title="Short film">short</a> <a href="/wiki/Western_(genre)" title="Western (genre)">Western film</a> written, produced, and directed by <a href="/wiki/Edwin_S._Porter" title="Edwin S. Porter">Edwin S. Porter</a>, a former <a href="/wiki/Edison_Studios" title="Edison Studios">Edison Studios</a> cameraman. Actors in the movie included <a href="/wiki/Alfred_C._Abadie" title="Alfred C. Abadie">Alfred C. Abadie</a>, <a href="/wiki/Broncho_Billy_Anderson" title="Broncho Billy Anderson">Broncho Billy Anderson</a> and <a href="/wiki/Justus_D._Barnes" title="Justus D. Barnes">Justus D. Barnes</a>, although there were no credits. Though a Western, it was filmed in <a href="/wiki/Milltown,_New_Jersey" title="Milltown, New Jersey">Milltown, New Jersey</a>. The film was inspired by <a href="/wiki/Scott_Marble" title="Scott Marble">Scott Marble</a>'s 189

In [83]:
soup.select('Navigation menu')   #.find_parent('h2').find_next_sibling()

[]

In [87]:
soup.find('h2').get_text()

'Contents'

In [89]:
soup.find('h1').get_text()

'The Great Train Robbery (1903 film)'

In [29]:
pprint.pprint(soup)

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Assassin (1973 film) - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XcSdKwpAAEIAAHuM1qsAAAAW","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Assassin_(1973_film)","wgTitle":"Assassin (1973 film)","wgCurRevisionId":724729741,"wgRevisionId":724729741,"wgArticleId":28833649,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from June 2016","Use British English from June 